In [1]:
import pandas as pd
import os
import re

# Selection of the files

Inside of the folder there is a file for each year. Thus in this part of the code working files are selected based on their name

In [2]:
folder_path = '006_population'
pattern_name = '.*S0201_data_with_overlays_*'

In [3]:
files_data = [(int(x.split('.')[0][-4:]), x) for x in os.listdir(folder_path) if re.match(pattern_name, x)]

In [4]:
files_data

[(2017, 'ACSSPP1Y2017.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2012, 'ACSSPP1Y2012.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2010, 'ACSSPP1Y2010.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2015, 'ACSSPP1Y2015.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2018, 'ACSSPP1Y2018.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2019, 'ACSSPP1Y2019.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2014, 'ACSSPP1Y2014.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2011, 'ACSSPP1Y2011.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2013, 'ACSSPP1Y2013.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2016, 'ACSSPP1Y2016.S0201_data_with_overlays_2022-06-05T101429.csv')]

# Ordering files

The order of the files is made based on the year (Ascending)

## Extract years and sort them

In [5]:
year_sorted = [year[0] for year in files_data]
year_sorted.sort()

## Sorting files
Based on the sorted year list, the files were sorted

In [6]:
files_data_sorted = [i for j in year_sorted for i in filter(lambda k: k[0] == j, files_data)]

In [7]:
files_data_sorted 

[(2010, 'ACSSPP1Y2010.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2011, 'ACSSPP1Y2011.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2012, 'ACSSPP1Y2012.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2013, 'ACSSPP1Y2013.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2014, 'ACSSPP1Y2014.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2015, 'ACSSPP1Y2015.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2016, 'ACSSPP1Y2016.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2017, 'ACSSPP1Y2017.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2018, 'ACSSPP1Y2018.S0201_data_with_overlays_2022-06-05T101429.csv'),
 (2019, 'ACSSPP1Y2019.S0201_data_with_overlays_2022-06-05T101429.csv')]

# Extract info for each file

A prior selection of the features was made and store in the variable features

In [8]:
features = {
    "id": "GEO_ID",
    "State": "NAME",
    "Total population": "S0201_001E",
    "Population in households": "S0201_038E",
    "Educational attainment": ["S0201_090E", "S0201_091E", "S0201_092E","S0201_093E", "S0201_094E", "S0201_095E"],
    "Employment status": ["S0201_154E", "S0201_155E","S0201_157E", "S0201_158E",  "S0201_161E"],
    "Income": ["S0201_213E", "S0201_214E", "S0201_226E", "S0201_227E"],
    "Housing ternure": ["S0201_265E", "S0201_266E", "S0201_267E"],
    "Housing cost": ["S0201_298E", "S0201_299E", "S0201_304E"]    
}

From the previous selection, a list was created to extract the info for each file using the pandas module

In [9]:
list_features = []
for key, value in features.items():
    if isinstance(value, list):
        list_features.extend(value)
    else:
        list_features.append(value)

Extraction the information

In [14]:
list_dataframes = []
for file_tuple in files_data_sorted:
    year, file_name = file_tuple
    df = pd.read_csv(folder_path + '/' + file_name)
    try:
        for feature in list_features:
            if feature not in df.columns:
                df[feature] = None
        df = df[list_features]
        df.set_index('GEO_ID', inplace=True)
        if '0400000US72' in df.index:
            df = df.drop(['id', '0400000US72'])
        else:
            df = df.drop('id')
        df = df.drop('NAME', axis=1)
        df = df.transpose()
        df['Year'] = year
        list_dataframes.append(df)
    except:
        print(file_name)
    
data = pd.concat(list_dataframes)
    

In [15]:
data

GEO_ID,0400000US01,0400000US02,0400000US04,0400000US05,0400000US06,0400000US08,0400000US09,0400000US10,0400000US11,0400000US12,...,0400000US47,0400000US48,0400000US49,0400000US50,0400000US51,0400000US53,0400000US54,0400000US55,0400000US56,Year
S0201_001E,4785298,713985,6413737,2921606,37349363,5049071,3577073,899769,604453,18843326,...,6356897,25257114,2776469,625960,8024617,6744496,1853973,5691047,564460,2010
S0201_038E,4669482,687633,6274353,2842675,36529547,4933193,3458921,875356,564432,18421617,...,6203425,24675975,2730317,600631,7784783,6605121,1804591,5540833,550748,2010
S0201_090E,3161521,450577,4146758,1919422,24097200,3328045,2434383,600779,416824,13077179,...,4242391,15772122,1584824,431042,5361301,4505657,1294747,3800295,370951,2010
S0201_091E,17.9,9.0,14.4,17.1,19.3,10.3,11.4,12.3,12.6,14.5,...,16.4,19.3,9.4,9.0,13.5,10.2,16.8,9.9,7.7,2010
S0201_092E,31.3,25.3,25.1,34.6,20.8,22.8,28.2,32.7,20.3,29.9,...,33.5,25.6,24.5,31.5,25.4,23.8,41.6,33.3,30.2,2010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
S0201_266E,68.8,64.7,65.3,65.5,54.9,65.9,65.0,70.3,41.5,66.2,...,66.5,61.9,70.6,70.9,66.1,63.1,73.4,67.2,71.9,2019
S0201_267E,31.2,35.3,34.7,34.5,45.1,34.1,35.0,29.7,58.5,33.8,...,33.5,38.1,29.4,29.1,33.9,36.9,26.6,32.8,28.1,2019
S0201_298E,154000,281200,255900,136200,568500,394600,280700,261700,646500,245100,...,191900,200400,330300,233200,288800,387600,124600,197200,235200,2019
S0201_299E,1172,1882,1457,1094,2421,1845,2087,1557,2684,1530,...,1264,1675,1605,1606,1792,1951,1052,1412,1417,2019


In [25]:

data.to_csv('project_data.csv')